In [2]:
# clone repository
!git clone https://github.com/Shounak-Ghosh/Adversarial_Examples_Results_Reproduction.git

Cloning into 'Adversarial_Examples_Results_Reproduction'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 17300 (delta 43), reused 19 (delta 9), pack-reused 17221
Receiving objects: 100% (17300/17300), 107.42 MiB | 13.17 MiB/s, done.
Resolving deltas: 100% (89/89), done.
Checking out files: 100% (17093/17093), done.


In [3]:
import scipy.io
import glob
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import keras.backend as K
from keras.models import load_model
from keras.utils.np_utils import to_categorical
from keras import metrics

import tensorflow as tf
from scipy.io import loadmat
from sklearn.metrics import confusion_matrix

plt.rcParams['figure.dpi'] = 300
%config InlineBackend.figure_format = 'retina'

In [6]:
!pwd
%cd Adversarial_Examples_Results_Reproduction

/content
/content/Adversarial_Examples_Results_Reproduction


In [7]:
#STEP 1: LETS VISUALIZE THE DATASET (ECG FILES)

# Load all the files with .mat extension
dataDir = './training2017/'
files = sorted(glob.glob(dataDir+"*.mat"))
print(f"The total number of ECG files is:{len(files)}")

The total number of ECG files is:8528


In [15]:
data_labels=pd.read_csv('revised_label.csv', header=None)
print(data_labels.head(15))
print("...........")

"""
Since we have no header in the labels, header = None
The dataset it is listed as just filename and label
Hence, 8528 data points (entries) are shown, no value is missing
"""

print(f"The shape of labels is{data_labels.shape}")
print("...........")
data_labels.info()

         0  1
0   A00001  N
1   A00002  N
2   A00003  N
3   A00004  A
4   A00005  A
5   A00006  N
6   A00007  N
7   A00008  O
8   A00009  A
9   A00010  N
10  A00011  N
11  A00012  N
12  A00013  O
13  A00014  N
14  A00015  A
...........
The shape of labels is(8528, 2)
...........
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8528 entries, 0 to 8527
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       8528 non-null   object
 1   1       8528 non-null   object
dtypes: object(2)
memory usage: 133.4+ KB


In [12]:
data_A=pd.read_csv('data_select_A.csv', header=None)
print(data_A.head(15))
print("...........")

"""
Since we have no header in the labels, header = None
The dataset it is listed as just filename and label
Hence, 8528 data points (entries) are shown, no value is missing
"""

print(f"The shape of labels is{data_A.shape}")
print("...........")
data_A.info()

      0         1          2      3
0   0.0  0.486398  30.000000    4.0
1   0.0  0.835322  60.000000    5.0
2   0.0  0.694854  30.000000    9.0
3   0.0  0.990192  30.000000   71.0
4   0.0  0.677061  30.000000   90.0
5   0.0  0.829382  30.000000  101.0
6   0.0  0.546336  30.000000  102.0
7   0.0  0.905562  23.426667  128.0
8   0.0  0.661574  30.000000  137.0
9   0.0  0.747092  30.000000  208.0
10  0.0  0.815178  30.000000  216.0
11  0.0  0.847697  30.000000  225.0
12  0.0  0.888139  60.000000  231.0
13  0.0  0.902549  30.000000  253.0
14  0.0  0.889798  30.000000  267.0
...........
The shape of labels is(447, 4)
...........
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447 entries, 0 to 446
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       447 non-null    float64
 1   1       447 non-null    float64
 2   2       447 non-null    float64
 3   3       447 non-null    float64
dtypes: float64(4)
memory usage: 14.1 KB


In [14]:
data_N=pd.read_csv('data_select_N.csv', header=None)
print(data_N.head(15))
print("...........")

"""
Since we have no header in the labels, header = None
The dataset it is listed as just filename and label
Hence, 8528 data points (entries) are shown, no value is missing
"""

print(f"The shape of labels is{data_N.shape}")
print("...........")
data_N.info()

      0         1          2     3
0   1.0  0.763950  30.000000   1.0
1   1.0  0.581624  30.000000   2.0
2   1.0  0.652977  60.000000   3.0
3   1.0  0.939361  30.000000   6.0
4   1.0  0.715151  30.000000   7.0
5   1.0  0.986109  60.000000  10.0
6   1.0  0.793361  30.000000  12.0
7   1.0  0.378632  18.566667  14.0
8   1.0  0.571122  30.000000  16.0
9   1.0  0.925395  30.000000  18.0
10  1.0  0.811436  38.400000  19.0
11  1.0  0.949535  30.000000  21.0
12  1.0  0.873536  30.000000  25.0
13  1.0  0.960950  30.000000  26.0
14  1.0  0.666901  60.000000  28.0
...........
The shape of labels is(3886, 4)
...........
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3886 entries, 0 to 3885
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       3886 non-null   float64
 1   1       3886 non-null   float64
 2   2       3886 non-null   float64
 3   3       3886 non-null   float64
dtypes: float64(4)
memory usage: 121.6 KB


In [16]:
data_O=pd.read_csv('data_select_O.csv', header=None)
print(data_O.head(15))
print("...........")

"""
Since we have no header in the labels, header = None
The dataset it is listed as just filename and label
Hence, 8528 data points (entries) are shown, no value is missing
"""

print(f"The shape of labels is{data_O.shape}")
print("...........")
data_O.info()

      0         1          2     3
0   2.0  0.892110  60.000000   8.0
1   2.0  0.806196  30.000000  13.0
2   2.0  0.588554  30.000000  20.0
3   2.0  0.638302  30.000000  23.0
4   2.0  0.741527  30.000000  29.0
5   2.0  0.664056  60.000000  30.0
6   2.0  0.800809  30.000000  38.0
7   2.0  0.690332  30.000000  41.0
8   2.0  0.802863  30.000000  61.0
9   2.0  0.977234  27.136667  65.0
10  2.0  0.603989  30.000000  69.0
11  2.0  0.886319  30.000000  74.0
12  2.0  0.607586  30.000000  77.0
13  2.0  0.746017  30.000000  82.0
14  2.0  0.616775  30.000000  88.0
...........
The shape of labels is(1488, 4)
...........
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1488 entries, 0 to 1487
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1488 non-null   float64
 1   1       1488 non-null   float64
 2   2       1488 non-null   float64
 3   3       1488 non-null   float64
dtypes: float64(4)
memory usage: 46.6 KB


In [17]:
data_i=pd.read_csv('data_select_i.csv', header=None)
print(data_i.head(15))
print("...........")

"""
Since we have no header in the labels, header = None
The dataset it is listed as just filename and label
Hence, 8528 data points (entries) are shown, no value is missing
"""

print(f"The shape of labels is{data_i.shape}")
print("...........")
data_i.info()

      0         1          2      3
0   3.0  0.906472  30.000000   22.0
1   3.0  0.848286  30.000000   34.0
2   3.0  0.960670  30.000000   56.0
3   3.0  0.892694  16.826667  106.0
4   3.0  0.947857  30.000000  125.0
5   3.0  0.994586  12.533333  139.0
6   3.0  0.978388  60.000000  164.0
7   3.0  0.987286  15.226667  201.0
8   3.0  0.381910  30.000000  205.0
9   3.0  0.830890  27.193333  307.0
10  3.0  0.975142  30.000000  370.0
11  3.0  0.983529  10.880000  445.0
12  3.0  0.997221  17.546667  474.0
13  3.0  0.914182  30.000000  524.0
14  3.0  0.876299  30.000000  537.0
...........
The shape of labels is(260, 4)
...........
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       260 non-null    float64
 1   1       260 non-null    float64
 2   2       260 non-null    float64
 3   3       260 non-null    float64
dtypes: float64(4)
memory usage: 8.2 KB
